<a href="https://colab.research.google.com/github/AlexxxSem111/quest/blob/main/Mistral_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U accelerate bitsandbytes

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    device_map="auto",
)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [21]:
def generate_response(query):
    messages = [
        {"role": "user", "content": query}
    ]

    input_encoding = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')

    generated_ids = model.generate(input_encoding,
                                   pad_token_id=tokenizer.eos_token_id,
                                   max_new_tokens=1000,
                                   do_sample=True)

    decoded_response = tokenizer.batch_decode(generated_ids,
                                              skip_special_tokens=True)

    #decoded_result = re.sub(r'\[INST\].*?\[/INST\] ', '', decoded_response[0])
    decoded_result = decoded_response[0].replace("[INST]", "Query:\n\n").replace("[/INST]", "\n\nStory:\n\n")

    return decoded_result

In [20]:
message = "tell me story 4-5 sentences about beer"

print(generate_response(message))

Query:

 tell me story 4-5 sentences about beer 

Story:

Once upon a time in Bavaria, Germany, a wealthy monk discovered the art of brewing beer. His creation, a delicious and refreshing concoction of malted barley, hops, water, and yeast, soon became a monastic tradition. The monks developed it into an intricate process, aging the beer in cool, dark cellars. This brew, known as Oktoberfestbier, was first served in 1810 to celebrate the wedding of the Bavarian Crown Prince, and the festival has continued every year since. Today, beer lovers gather from around the world to savor this rich, malty elixir, raising a stein and making new friendships.
